<a href="https://colab.research.google.com/github/ElisaCastelli/CPAC_CollectiveDynamicPortrait/blob/main/CPACprovaFaceDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style transfer

In [57]:
# explain various inputs
import numpy as np
import PIL


import tensorflow_hub as hub
import tensorflow as tf

In [2]:
# explain what does this function do
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [3]:
# Converts tensor into image
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

### organize style files

In [67]:
image_format = '.png'
# here I should receive parameters from spotify
acousticness = 0
valence = 0

#style_filename =  str(acousticness) + str(valence) + image_format
style_filename = 'yellow_cables.jpg'
style_filename

'yellow_cables.jpg'

In [72]:
content_image = load_img('style_pics/dummy_photo.jpg')
style_image = load_img(style_filename)

NotFoundError: NewRandomAccessFile failed to Create/Open: yellow_cables.jpg : The system cannot find the file specified.
; No such file or directory [Op:ReadFile]

In [69]:
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1')
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
output = tensor_to_image(stylized_image)

output.save("style_pics/dummy_photo11.jpg") 

tensor_to_image(stylized_image)

InvalidArgumentError: cannot compute __inference_pruned_139327 as input #1(zero-based) was expected to be a float tensor but is a string tensor [Op:__inference_pruned_139327]